<a href="https://colab.research.google.com/github/ykitaguchi77/GAN/blob/master/ChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount("/content/drive")

!pip install openai --q
!pip install --upgrade deepl --q
import openai
import deepl
import textwrap
import pandas as pd

with open("/content/drive/MyDrive/Deep_learning/api.txt") as file:
    #text = file.read()
    i=1
    key = []
    while True:
        include_break_line = file.readline() #改行が含まれた行
        line = include_break_line.rstrip() #改行を取り除く
        if line: #keyの読み込み
            #print(f'{i}行目：{line}')
            key.append(line)
            i += 1
        else:
            break

# APIキーの準備
#ngrok_aceess_token = key[5]
openai.api_key = key[3]
deepl_auth_key = key[1]

translator = deepl.Translator(deepl_auth_key)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
#@title #**ChatGPT configulation**
question_translate_English= True #@param {type:"boolean"}
answer_Japanese= False #@param {type:"boolean"}
reset= True #@param {type:"boolean"}
show_all = True #@param {type:"boolean"}
arrangement = True #@param {type:"boolean"}



if "text_input" not in globals():
    text_input = []
if reset == True:
    text_input = []


def wrap(sentence):
    wrap_list =  textwrap.wrap(sentence, 120)
    output = '\n'.join(wrap_list)
    return(output)


def interference(question):
    question = question.replace('\n', '') #前後の空白と改行を削除

    if question_translate_English == True:
        question_translated = translator.translate_text(question, target_lang="EN-US").text
        question = question_translated
    else:
        question = question

    prompt = "Q:" + question +"\nA:"

    text_input.append(question+"\nA:")

    prompt =  ",".join(map(str, text_input)) # aaa,bbb,100,200,300 

    response = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt,
      temperature=0.5,
      max_tokens= 1024,
      #top_p=1,
      frequency_penalty=0.0,
      presence_penalty=0.0,
      stop=None)

    answer = response['choices'][0]['text']
    text_input.append(answer)

    if answer_Japanese == True:
        answer = translator.translate_text(response['choices'][0]['text'], target_lang="JA").text
    else:
        answer = answer

    output(text_input, show_all=show_all, arrangement=arrangement)
    # print(f"{prompt}")
    # print(f"{wrap(answer)} \n")
    #print(text_input)


def output(text_input, show_all = True, arrangement = False):
    if arrangement == True:
        if show_all == False:
            print(f"Q: {wrap(text_input[-2]).rstrip('A:')} \n")
            print(f"A: {wrap(text_input[-1])} \n")

        elif show_all == True:
            for i, text in enumerate(text_input):
                if i % 2 == 0:
                    print(f"Q: {wrap(text).rstrip('A:')} \n")
                else:
                    print(f"A: {wrap(text)} \n")

    elif arrangement == False:
        if show_all == False:
            print(f"Q: {text_input[-2].rstrip('A:')} \n")
            print(f"{text_input[-1]} \n")

        elif show_all == True:
            for i, text in enumerate(text_input):
                if i % 2 == 0:
                    print(f"Q: {text.rstrip('A:')} \n")
                else:
                    print(f"{text} \n")




In [ ]:
#@title #**Question & Answer list**
Leave_first_2= False #@param {type:"boolean"}
Leave_last_2 = False #@param {type:"boolean"}
Leave_last_4 = False #@param {type:"boolean"}
Leave_last_6 = False #@param {type:"boolean"}

tmp = []

if Leave_first_2 == True:
    tmp = [i for i in text_input[0:2]]    

if Leave_last_6 == True:
    tmp.extend([i for i in text_input[-6:]])
elif Leave_last_4 == True:
    tmp.extend([i for i in text_input[-4:]])
elif Leave_last_2 == True:
    tmp.extend([i for i in text_input[-2:]])

text_input = [text_input[i] for i in tmp]

print(text_input)

In [14]:
Delete_last_two= True #@param {type:"boolean"}
if Delete_last_two == True:
    text_input = text_input[:-2]

output(text_input)

['I need you to act as a research assistant for a scientific article I am writing. I will provide you with specific questions and information, and I expect you to find relevant and credible sources to support my research. Your responses should include a summary of the main points from the sources, along with the full citation in the desired format (APA, MLA, etc.). Additionally, you should flag any potential issues or limitations in the sources you provide. The title of the article is: Real-time smartphone identification of corneal diseases displayed on computer screen using embedded deep-learning model. Please show me the bullets which I need to contain in the introduction.\nA:', ' \n• Introduction to corneal diseases and their diagnosis \n• Overview of current methods for corneal disease diagnosis \n• Description of the use of embedded deep-learning models for real-time smartphone identification of corneal diseases \n• Overview of the advantages and disadvantages of using deep-learni

#**Interference**

In [16]:
question = '''

Please wrihte the first paragraph in detail, Introduction to corneal diseases and their diagnosis, including the consequences of these diseases if left untreated

'''
interference(question)

Q: I need you to act as a research assistant for a scientific article I am writing. I will provide you with specific
questions and information, and I expect you to find relevant and credible sources to support my research. Your responses
should include a summary of the main points from the sources, along with the full citation in the desired format (APA,
MLA, etc.). Additionally, you should flag any potential issues or limitations in the sources you provide. The title of
the article is: Real-time smartphone identification of corneal diseases displayed on computer screen using embedded
deep-learning model. Please show me the bullets which I need to contain in the introduction.  

A:   • Introduction to corneal diseases and their diagnosis  • Overview of current methods for corneal disease diagnosis  •
Description of the use of embedded deep-learning models for real-time smartphone identification of corneal diseases  •
Overview of the advantages and disadvantages of using deep-learning m

In [ ]:
output(text_input, show_all = False, arrangement = True)

Q: Corneal disease is a common and debilitating condition that affects the transparency of the clear front surface of the
eye. If left untreated, some corneal diseases, especially corneal infections and autoimmune diseases, can lead to
permanent corneal opacity and vision loss. On the contrary, the consequences of these diseases are mostly preventable if
treated promptly. Therefore, early diagnosis and treatment of corneal diseases are essential to prevent progression of
the disease and its associated complications.Slit lamp examination is a key element in the diagnosis of corneal disease.
It allows the practitioner to observe the cornea, lens, iris and other structures of the eye in detail. However, slit
lamp examination has the limitation that the diagnostic performance is largely dependent on the skill of the
practitioner. Since not all institutions have specialists in corneal diseases, it is critical to secure resources to
avoid missing emerging diseases.Next, I want to write overv

# **Save and load data**

In [ ]:
save_dir = "/content/drive/MyDrive/Deep_learning/chatGPT"
file_name = "corneAI"


# save_data
import pickle
f = open(f'{save_dir}/{file_name}.txt', 'wb')
list_row = text_input
pickle.dump(list_row, f)

In [ ]:
# load data
f = open(f'{save_dir}/{file_name}.txt', 'rb')
text_input = pickle.load(f)